# Chunking and SQLite

In this notebook, we'll see a couple of techniques that can be used whe working with large file in Python.

Specifically, we'll be looking at the dataset of Metro Nashville Police Department Calls for Service (https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Calls-for-Servic/kwnd-qrrm) with the goal being to look at specific types of calls for service and to bring in additional information, when possible, from the Metro Nashville Police Department Incidents file (https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs).

In [1]:
import pandas as pd

We can start by reading in just a few rows to get an idea of what the dataset we are working with looks like. This can be done using the `nrows` argument.

In [2]:
metro_calls = pd.read_csv('../data/Metro_Nashville_Police_Department_Calls_for_Service_20240222.csv', 
                          nrows = 100)

metro_calls.head()

,Event Number,Call Received,Complaint Number,Tencode,Tencode Description,Tencode Suffix,Tencode Suffix Description,Disposition Code,Disposition Description,Block,Street Name,Unit Dispatched,Shift,Sector,Zone,RPA,Latitude,Longitude,Mapped Location
0,PD202100625493,10/02/2021 07:13:21 PM,NaN,71,BURGLARY - NON-RESIDENCE,A,ALARM,11,DISREGARD / SIGNAL 9,NaN,NaN,NaN,B,H,523,9557.0,NaN,NaN,NaN
1,PD202100626631,10/03/2021 11:25:42 AM,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,4,ASSISTED CITIZEN,NaN,NaN,513A,A,H,513,9033.0,NaN,NaN,NaN
2,PD202100626031,10/03/2021 12:54:02 AM,NaN,46,VEHICLE ACCIDENT - PERSONAL INJURY,P,PROGRESS,11,DISREGARD / SIGNAL 9,NaN,NaN,NaN,C,S,323,8889.0,NaN,NaN,NaN
3,PD202100622285,10/01/2021 08:05:43 AM,NaN,43,WANT OFFICER FOR INVESTIGATION / ASSISTA,P,PROGRESS,5,GONE ON ARRIVAL,200.0,OLD HICKORY BLVD,123A,A,NaN,136W,0.0,36.079,-86.919,POINT (-86.919 36.079)
4,PD202100621219,09/30/2021 06:48:20 PM,2.021051e+10,40,SUSPICIOUS PERSON,TS,TERRY STOP,6,ASSISTED OTHER UNIT,NaN,NaN,2P63,B,NaN,NaN,NaN,NaN,NaN,NaN


Let's say that our goal is to find all incidents where the Tencode Description was 'SHOTS FIRED'.

One thing we could try is to use the chunksize argument in our pd.read_csv call. What this does is to create an iterable which returns just the specified number of rows at a time.

Iterating through a file using chunks can look like this, but can also be structured differently (for example, using a list comprehension).

``` 
chunks = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)

for chunk in chunks:
    # Do something 
```

Here is what we need to do:

1. Create an iterable by using the chunksize argument.

2. For each chunk, filter to just the rows where the 'Tencode Description' column is 'SHOTS FIRED'. Store these rows.

3. Concatenate all the results together into a single dataframe.

In [5]:
chunks = pd.read_csv('../data/Metro_Nashville_Police_Department_Calls_for_Service_20240222.csv', 
                     chunksize = 10000)

shots_fired = []

for chunk in chunks:
    chunk = chunk[chunk['Tencode Description'] == 'SHOTS FIRED']
    shots_fired.append(chunk)
    
shots_fired = pd.concat(shots_fired, ignore_index = True)

In [7]:
shots_fired.shape

(53856, 19)

The above solution would work for one-off tasks. However, if you are going to be working extensively with a dataset or merging two large datasets, it would be a bit cumbersome to have to chunk through one or both datasets multiple times.

As an alternative to working in just Python, we can use a different tool which works better on large datasets - SQL.

In this notebook, we will make use of SQLite, which is a file-based relational database management system. We can interact with SQLite databases through the sqlite3 library.

**Note:** You only need to set up a database once! After that, you can just connect to it and pull in the data you need.

In [8]:
import sqlite3

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [9]:
db = sqlite3.connect('../data/police_calls.sqlite')

for chunk in pd.read_csv('../data/Metro_Nashville_Police_Department_Calls_for_Service_20240222.csv', 
                              chunksize = 10000):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('calls', 
                 db, 
                 if_exists = 'append', 
                 index = False)            # Append the chunk to a calls table

To speed up queries which use a specific column, we can create an **index** on that column. This causes the database to store that column in a way that helps it to retrieve rows quicker.

In [10]:
db.execute('CREATE INDEX tencode_description ON calls(tencode_description)')

Finally, we should close our database connection.

In [11]:
db.close()

Now, let's see how long it takes to find all rows corresponding to 'SHOTS FIRED'.

In [12]:
db = sqlite3.connect('../data/police_calls.sqlite')

query = "SELECT * FROM calls WHERE tencode_description = 'SHOTS FIRED'"

shots_sqlite = pd.read_sql(query, db)

db.close()

In [13]:
shots_sqlite.shape

(53856, 19)

You can also use a **context manager** which will take care of closing the connection for you.

In [ ]:
query = "SELECT * FROM calls WHERE tencode_description = 'SHOTS FIRED'"

with sqlite3.connect('../data/police_calls.sqlite') as db: 
    shots_sqlite = pd.read_sql(query, db)

We can also do things like find the most frequent tencode.

In [14]:
query = """
SELECT tencode_description, COUNT(*) AS num_calls
FROM calls
GROUP BY tencode_description
ORDER BY num_calls DESC
"""

with sqlite3.connect('../data/police_calls.sqlite') as db:
    counts = pd.read_sql(query, db)
    
counts.head()

,tencode_description,num_calls
0,WANT OFFICER FOR INVESTIGATION / ASSISTA,1063455
1,BUSINESS CHECK,813425
2,TRAFFIC VIOLATION,604989
3,THEFT,280775
4,COMMUNITY POLICING ACTIVITY,277399


Now, let's add in the Metro Police Department Incidents database.

In [16]:
db = sqlite3.connect('../data/police_calls.sqlite')

for chunk in pd.read_csv('../data/Metro_Nashville_Police_Department_Incidents_20240222.csv', 
                              chunksize = 10000):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('incidents', db, if_exists = 'append', index = False)            # Append the chunk to a calls table
    
db.close()

If we want to match calls to incidents, we can use the complaint_number column from the calls database and the incident_number column from the incidents database.

To speed up this process, we can created indexes on these two columns.

In [17]:
db = sqlite3.connect('../data/police_calls.sqlite')

db.execute('CREATE INDEX complaint_number ON calls(complaint_number)')
db.execute('CREATE INDEX incident_number ON incidents(incident_number)')

db.close()

Now, let's grab all SHOTS FIRED calls for which there is an associated incident and bring in the incident information. Note that the complaint_number from calls matches the incident_number from incidents.

In [ ]:
query = """
SELECT * FROM calls AS c 
JOIN incidents AS i 
ON c.complaint_number = i.incident_number 
WHERE tencode_description = 'SHOTS FIRED'
"""

with sqlite3.connect('../data/police_calls.sqlite') as db: 
    shots_sqlite = pd.read_sql(query, db)

In [ ]:
shots_sqlite